
# Instalação
Instalação de bibliotecas

In [ ]:
#Instalação do groq
%pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 2.1 MB/s eta 0:00:00


In [ ]:
#Impostação da Chave da API encontrada no groq
import os
os.environ['GROQ_API_KEY'] = "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"

In [ ]:
#Definição do client e modelo llm a ser usado
#O cliente serve para interagir com a API da Groq
import os
from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models", #Conteúdo da mensagem a ser gerada pelo modelo
        }
    ],
    model="llama3-70b-8192", #Definição do modelo escolhido(llama-70b-8192)
)
#Exibe a mensagem gerada pelo modelo
print(chat_completion.choices[0].message.content)

Fast language models are essential in natural language processing (NLP) and have numerous applications in various industries. Here are some reasons why fast language models are important:

1. **Real-time Processing**: Fast language models enable real-time processing of vast amounts of text data, which is crucial in applications like chatbots, virtual assistants, and sentiment analysis. They can quickly analyze and respond to user inputs, providing a seamless user experience.
2. **Efficient Resource Utilization**: Fast language models require fewer computational resources, which leads to significant cost savings and reduced environmental impact. They can run on lower-end hardware, making them more accessible to organizations with limited resources.
3. **Scalability**: Fast language models can handle large volumes of text data, making them ideal for applications that involve processing massive datasets, such as language translation, text summarization, and information retrieval.
4. **Low

In [ ]:
#Definição dos defs do agente
class Agent: #Agent é uma classe que representa um agente de IA interagindo com um modelo de linguagem.
  def __init__(self, client, system):
    self.client = client
    self.system = system
    self.messages = [] #Armazena mensagens anteriores para manter o contexto da conversa.
    if self.system is not None: #Se um system_prompt for definido, ele será adicionado como mensagem do sistema
      self.messages.append({"role": "system", "content": self.system})

  def __call__(self, message=""): #Permite chamar o agente diretamente
    if message:
        self.messages.append({"role": "user", "content": message}) #Adiciona a mensagem do usuário ao histórico
    result = self.execute() #Chama o método execute()
    self.messages.append({"role": "assistant", "content": result})
    return result

  def execute(self): #Executa a solicitação para o modelo de linguagem e retorna a resposta
      completion = client.chat.completions.create(
          messages=self.messages,
          model="llama-3.3-70b-versatile", #Usa o modelo "llama3-70b-8192" para responder com base no contexto armazenado
      )
      return completion.choices[0].message.content #Retorna a resposta

In [ ]:
#Prompt para o agente React, utilizando alturas de torres
#este prompt define o ciclo a ser seguido de Thought, Action, PAUSE, Observation
#Define as ferramentas disponíveis para o agente
#Da um exemplo para melhorar a compreensão do agente
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.
Use Thought to describe your reasoning about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

get_tower_height:
e.g. get_tower_height: Eiffel Tower
Returns the height of the tower in meters.

calculate:
e.g. calculate: 2 * 330
Runs a calculation and returns the result.

Example session:

Question: What is twice the height of the Eiffel Tower?
Thought: I need to get the height of the Eiffel Tower.
Action: get_tower_height: Eiffel Tower
PAUSE

You will be called again with this:

Observation: 330

Thought: I need to multiply that height by 2.
Action: calculate: 2 * 330
PAUSE

You will be called again with this:

Observation: 660

If you have the answer, output it as the Answer.

Thought: I have the final answer.
Answer: Twice the height of the Eiffel Tower is 660 meters.

Now it's your turn:
""".strip()


In [ ]:
#define a função para realizar operações matemáticas
def calculate(operation): #Operações realizadas por meio de eval()
    return eval(operation)

#define a função para retornar a altura de torres famosas em metros
def get_tower_height(tower_name): #Retorna a altura de torres famosas em metros
    tower_heights = {
        "eiffel tower": 330,
        "burj khalifa": 828,
        "empire state building": 381,
        "tokyo tower": 332,
        "cn tower": 553
    }
    return tower_heights.get(tower_name.lower(), "Height not found")


In [ ]:
#impostação do Agente com base na classe e definição de seu nome
luana = Agent(client=client, system=system_prompt)

In [ ]:
#Rodando por uma vez o agente para verificar se está funcionando
result = luana("What is the height of eiffel tower times 5?")
print(result)

Thought: To find the height of the Eiffel Tower times 5, I first need to get the height of the Eiffel Tower.
Action: get_tower_height: Eiffel Tower
PAUSE


In [ ]:
#criaçãodo loop que irá rodar o ciclo múltiplas vezes até alcançar uma resposta satisfatória
#Permite automatizar o ciclo do agente, rodando múltiplas iterações
#Identifica ações sugeridas pelo modelo e executa funções (calculate ou get_planet_mass)
#Usa regex (re.findall()) para capturar a ação desejada pelo modelo
import re

def agent_loop(max_iterations, system, query):
  agent = Agent(client=client, system=system_prompt)
  tools = ['calculate', 'get_tower_height']
  next_prompt = query
  i = 0
  while i < max_iterations:
    i += 1
    result = agent(next_prompt)
    print(result)

    if "PAUSE" in result and "Action" in result:
      action = re.findall(r'Action: ([a-z_]+): (.+)', result, re.IGNORECASE)
      choosen_tool = action[0][0]
      arg = action[0][1]

      if choosen_tool in tools:
        result_tool = eval(f"{choosen_tool}('{arg}')")
        next_prompt = f"Observation: {result_tool}"

      else:
        next_prompt = "Observation: Tool not found"

      print(next_prompt)
      continue

    if "Answer" in result:
      break

In [ ]:
#Rodando o loop com os parametros desejados e inserindo a query
agent_loop(max_iterations=10, system=system_prompt, query="What is the height of burj khalifa plus the height of the tokyo tower, and all of it times 4")

Thought: To solve this problem, I need to get the heights of the Burj Khalifa and the Tokyo Tower, add them together, and then multiply the result by 4. First, I'll get the height of the Burj Khalifa.

Action: get_tower_height: Burj Khalifa
PAUSE
Observation: 828
Thought: Now that I have the height of the Burj Khalifa, I need to get the height of the Tokyo Tower to add them together.

Action: get_tower_height: Tokyo Tower
PAUSE
Observation: 332
Thought: I now have the heights of both the Burj Khalifa (828 meters) and the Tokyo Tower (332 meters). I'll add these two heights together.

Action: calculate: 828 + 332
PAUSE
Observation: 1160
Thought: I now have the sum of the heights of the Burj Khalifa and the Tokyo Tower, which is 1160 meters. The next step is to multiply this sum by 4 to get the final result.

Action: calculate: 1160 * 4
PAUSE
Observation: 4640
Thought: I have now calculated the sum of the heights of the Burj Khalifa and the Tokyo Tower multiplied by 4, which equals 4640 